![@mikegchambers](../../images/header.png)

# Recurrent Neural Networks

In this notebook, we explore Recurrent Neural Networks using TensorFlow and a custom dataset.  Let's predict some stock prices (not really).

![Stocks](stocks.png)

UPDATE: Select the `conda_tensorflow2_p310` kernel when prompted. 

In [ ]:
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# The Data

We have a CSV file in this folder. Let's load it now.  It has two columns, but we only want the 2nd column.  We also converyt it to flat values just incase.

In [ ]:
dataframe = pd.read_csv('data.csv', usecols=[1])
dataset = dataframe.values
dataset = dataset.astype('float32')

Let's have a look at the data.  We can see a pattern, it looks like this 'stock' price fluctuates with some regularity. :)

In [ ]:
plt.plot(dataset)
plt.show()

## Process the data

Like many algorithms (especially when dealing with multiple features of data) we want to standardize or normalize the data.  This means, in this case, mapping the values between 0 and 1. 

BUT we want to keep track of how we did this so we can reverse the process later when we make predictions.

To do this we are going to use `MinMaxScaler` from scikit-learn.  So this is yet another example of using multiple frameworks to solve a problem.

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_dataset = scaler.fit_transform(dataset)

Now to split the data.  Should be use scikit-learn `train_test_split`?

No.  `train_test_split` will trandomise the data, and in this case the ordering of the data is important.

Instead we can split the data with some simple Python.

We end up with training dataset which is 70% of our data, and the rest is test data. Later we will try and predict the test data and see how close we get.

In [ ]:
train_size = int(len(scaled_dataset) * 0.70)
test_size = len(scaled_dataset) - train_size

train, test = scaled_dataset[0:train_size,:], scaled_dataset[train_size:len(scaled_dataset),:]

To create the labels we create a labelset where a label for a given X is the next X in the dataset.  
e.g. X=t and y=t+1

In [ ]:
def create_dataset(datapoints, look_back=1):
    
    X = []
    y = []
    
    for i in range(len(datapoints)-look_back-1):
        
        a = datapoints[i:(i+look_back), 0]
        X.append(a)
        
        b = datapoints[i + look_back, 0]
        y.append(b)
        
    return np.array(X), np.array(y)

Use that function to create our labels

In [ ]:
look_back = 1
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

Now we need to get the data into a 'shape' that RNN expects.

That is n array of dimentions: samples, time steps, and features

In [ ]:
X_train_rnn = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn  = np.reshape(X_test,  (X_test.shape[0],  1, X_test.shape[1]))

# The Model

The definition of this model is simple enough.  We have one SimpleRNN layer.  Keras has layers for LSTM's as well, maybe try swapping that in later.

In [ ]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.SimpleRNN(8, input_shape=(1, look_back)))
# model.add(tf.keras.layers.LSTM(8, input_shape=(1, look_back)))
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='adam',
              loss='mean_squared_error')

Train our model and store the loss value over time.

In [ ]:
e = model.fit(X_train_rnn, y_train, batch_size=1, epochs=10)

Plot the loss graph over Epochs.

In [ ]:
plt.plot(e.history['loss'])
plt.show()

# Predictions

To test the model we will make a prediction each of the test values we have.  To be clear what this will do is for the last 30% of our original data, it will make a prediction for the next value in the dataset.

In [ ]:
X_test_predict = model.predict(X_test_rnn)

Remember when we scaled the data?  Well we need to inverse the scale on our predictions so that they match the 'real world' data.

In [ ]:
X_test_predict_rescaled = scaler.inverse_transform(X_test_predict)

Now we need to shift the 'time' of the new data so that it matches the position of the test data when we plot it.

In [ ]:
X_test_predict_plot = np.empty_like(scaled_dataset)
X_test_predict_plot[:, :] = np.nan
X_test_predict_plot[len(X_train)+(look_back*2)+1:len(scaled_dataset)-1, :] = X_test_predict_rescaled

How did we do?

In [ ]:
plt.plot(dataset, c='red')
plt.plot(scaler.inverse_transform(X_train), c='green')
plt.plot(X_test_predict_plot, c='blue')
plt.show()

Inspiration for some of the code here came from Jason Brownlee's article here: 
https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/